### 一. 与数据倾斜有关的基本概念

#### 1. 什么是数据倾斜
Spark这样的大数据系统, 不怕数据量大, 最害怕数据倾斜. 数据倾斜会导致某个分区的数据远远大于其他分区, 使得该部分的处理速度成为整个数据集处理的瓶颈  
#### 2. 数据倾斜是如何造成的
spark应用中, 同一个stage内的不同分区的数据可以并行处理. 如一个stage可以分成N个task, 每个task分别处理一个分区. 如果其中N-1个Task都在10秒内完成，而另外一个Task却耗时1分钟，那该Stage的总时间至少为1分钟。换句话说，一个Stage所耗费的时间，主要由最慢的那个Task决定。

#### 3. 分区数, task数量, executor数量的关系 
1. 1个task处理1个分区上的数据  
2. 1个executor由若干个core组成, 每个core一次执行一个task  
3. executor数量\*core数量 = task数量 = 输入分区的数量  
4. Map阶段partition数目保持不变。   
在Reduce阶段，RDD的聚合会触发shuffle操作，聚合后的RDD的partition数目跟具体操作有关，例如repartition操作会聚合成指定分区数，还有一些算子是可配置的

### 二. 处理数据倾斜的方法

#### 1. 调整并行度, 使原本送到一个分区的不同的key, 被分到不同的分区
1. 原理  
Spark在做Shuffle时，默认使用HashPartitioner（非Hash Shuffle）对数据进行分区。如果并行度设置的不合适，可能造成大量不相同的Key对应的数据被分配到了同一个Task上，造成该Task所处理的数据远大于其它Task，从而造成数据倾斜。如果调整Shuffle时的并行度，使得原本被分配到同一Task的不同Key发配到不同Task上处理，则可降低原Task所需处理的数据量，从而缓解数据倾斜问题造成的短板效应。  
2. 适用场景  
 适用于大量不同的key被分配到一个分区, 使得该分区的task处理的数据过大  
3. 劣势  
   无法解决同一个key倾斜严重的场景.使用场景少,因此效果一般
   
#### 2. 自定义Partitioner
1. 原理  
使用自定义的Partitioner（默认为HashPartitioner），将原本被分配到同一个Task的不同Key分配到不同Task。
```scala
.groupByKey(new Partitioner() {
  @Override
  public int numPartitions() {
    return 12;
  }

  @Override
  public int getPartition(Object key) {
    int id = Integer.parseInt(key.toString());
    if(id &gt;= 9500000 && id &lt;= 9500084 && ((id - 9500000) % 12) == 0) {
      return (id - 9500000) / 12;
    } else {
      return id % 12;
    }
  }
})
```
2. 适用场景  
大量不同的Key被分配到了相同的Task造成该Task数据量过大。  
3. 劣势  
适用场景有限，只能将不同Key分散开，对于同一Key对应数据集非常大的场景不适用。效果与调整并行度类似，只能缓解数据倾斜而不能完全消除数据倾斜。而且需要根据数据特点自定义专用的Partitioner，不够灵活。

#### 3. 将小表设置成Broadcast变量, 彻底避免join阶段的shuffle操作  
1. 思路  
 可以将Broadcast的变量的大小限制设置的足够大, 然后让小表自动变成广播变量发送给大表的每个executor, 彻底避免join时的shuffle.
2. 设置
```bash
SET spark.sql.autoBroadcastJoinThreshold=104857600;
INSERT OVERWRITE TABLE test_join
SELECT test_new.id, test_new.name
FROM test      # 大表
JOIN test_new  # 小表
ON test.id = test_new.id;
```
3. 适用场景   
参与Join的一边数据集足够小，可被加载进Driver并通过Broadcast方法广播到各个Executor中。

4. 优势  
避免了Shuffle，彻底消除了数据倾斜产生的条件，可极大提升性能。

5. 劣势  
要求参与Join的一侧数据集足够小，并且主要适用于Join的场景，不适合聚合的场景，适用条件有限。

#### 4. 为倾斜的key增加随机的前缀后缀
1. 适用场景  
2个rdd进行join, 1个rdd中有几个key倾斜, 另一个rdd的key分布均匀, 且两个表都很大的情况下的join
2. 解决方案   
扫面一遍数据倾斜的RDD, 将倾斜的key与不倾斜的key分开;将倾斜的key取出来加上随机前缀, 另一个不倾斜的RDD每条数据分别与这个随机前缀集合进行笛卡尔积形成新的RDD. (注:此时不倾斜的RDD数据膨胀k倍,k为随机前缀个数).然后二者进行join, 并删除结果RDD中key的随机前缀.再将不包含倾斜Key的剩余数据进行Join. 最后将两次Join的结果集通过union合并，即可得到全部Join结果。  
3. 劣势  
  如果倾斜Key非常多，则另一侧数据膨胀非常大，此方案不适用。而且此时对倾斜Key与非倾斜Key分开处理，需要扫描数据集两遍，增加了开销。